In [1]:
from pyspark import *

In [2]:
sc = SparkContext(master="local", appName="test")
sqlContext = SQLContext(sc)

In [3]:
sc

<SparkContext master=local appName=test>

In [10]:
import urllib
data = urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz', 'data_10_percent.gz')
data_file = './data_10_percent.gz'

In [116]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

lines = sc.textFile(data_file)
parts = lines.map(lambda l: l.split(","))
parts.take(1)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.']]

In [123]:
datas = parts.map(lambda key: (int(key[0]),key[1],key[2],key[3],int(key[4]),int(key[5]),key[-1]))
datas.take(10)

[(0, 'tcp', 'http', 'SF', 181, 5450, 'normal.'),
 (0, 'tcp', 'http', 'SF', 239, 486, 'normal.'),
 (0, 'tcp', 'http', 'SF', 235, 1337, 'normal.'),
 (0, 'tcp', 'http', 'SF', 219, 1337, 'normal.'),
 (0, 'tcp', 'http', 'SF', 217, 2032, 'normal.'),
 (0, 'tcp', 'http', 'SF', 217, 2032, 'normal.'),
 (0, 'tcp', 'http', 'SF', 212, 1940, 'normal.'),
 (0, 'tcp', 'http', 'SF', 159, 4087, 'normal.'),
 (0, 'tcp', 'http', 'SF', 210, 151, 'normal.'),
 (0, 'tcp', 'http', 'SF', 212, 786, 'normal.')]

In [124]:
from pyspark.sql.types import *

struc = StructType(
    [
        StructField("duration", IntegerType(), True),
        StructField("protocol_type", StringType(), True),
        StructField("service", StringType(), True),
        StructField("flag", StringType(), True),
        StructField("src_bytes", IntegerType(), True),
        StructField("dst_bytes", IntegerType(), True),
        StructField("interactions", StringType(), True)
    ]
)
struc

StructType(List(StructField(duration,IntegerType,true),StructField(protocol_type,StringType,true),StructField(service,StringType,true),StructField(flag,StringType,true),StructField(src_bytes,IntegerType,true),StructField(dst_bytes,IntegerType,true),StructField(interactions,StringType,true)))

In [125]:
df = sqlContext.createDataFrame(datas, struc)
df

DataFrame[duration: int, protocol_type: string, service: string, flag: string, src_bytes: int, dst_bytes: int, interactions: string]

In [126]:
df.columns

['duration',
 'protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'interactions']

In [127]:
df.show(10)

+--------+-------------+-------+----+---------+---------+------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|interactions|
+--------+-------------+-------+----+---------+---------+------------+
|       0|          tcp|   http|  SF|      181|     5450|     normal.|
|       0|          tcp|   http|  SF|      239|      486|     normal.|
|       0|          tcp|   http|  SF|      235|     1337|     normal.|
|       0|          tcp|   http|  SF|      219|     1337|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      212|     1940|     normal.|
|       0|          tcp|   http|  SF|      159|     4087|     normal.|
|       0|          tcp|   http|  SF|      210|      151|     normal.|
|       0|          tcp|   http|  SF|      212|      786|     normal.|
+--------+-------------+-------+----+---------+---------+------------+
only s

In [145]:
df.createOrReplaceTempView("interactions")
# Copie des données, en l'appelant interactions, pour travailler sur une copie des données sans les impactés

In [283]:
result = df[
    (df['protocol_type'] == 'tcp') 
    & 
    (df['duration']>1000) 
    & 
    (df['dst_bytes'] == 0)
].select(df['duration'], df['dst_bytes'])
result.show()
type(result)

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [169]:
result.count()

139

In [306]:
result.rdd.map(lambda r: 'Duration: {0} , Dest. bytes: {1}'.format(r['duration'], r['dst_bytes'])).take(result.count())

['Duration: 5057 , Dest. bytes: 0',
 'Duration: 5059 , Dest. bytes: 0',
 'Duration: 5051 , Dest. bytes: 0',
 'Duration: 5056 , Dest. bytes: 0',
 'Duration: 5051 , Dest. bytes: 0',
 'Duration: 5039 , Dest. bytes: 0',
 'Duration: 5062 , Dest. bytes: 0',
 'Duration: 5041 , Dest. bytes: 0',
 'Duration: 5056 , Dest. bytes: 0',
 'Duration: 5064 , Dest. bytes: 0',
 'Duration: 5043 , Dest. bytes: 0',
 'Duration: 5061 , Dest. bytes: 0',
 'Duration: 5049 , Dest. bytes: 0',
 'Duration: 5061 , Dest. bytes: 0',
 'Duration: 5048 , Dest. bytes: 0',
 'Duration: 5047 , Dest. bytes: 0',
 'Duration: 5044 , Dest. bytes: 0',
 'Duration: 5063 , Dest. bytes: 0',
 'Duration: 5068 , Dest. bytes: 0',
 'Duration: 5062 , Dest. bytes: 0',
 'Duration: 5046 , Dest. bytes: 0',
 'Duration: 5052 , Dest. bytes: 0',
 'Duration: 5044 , Dest. bytes: 0',
 'Duration: 5054 , Dest. bytes: 0',
 'Duration: 5039 , Dest. bytes: 0',
 'Duration: 5058 , Dest. bytes: 0',
 'Duration: 5051 , Dest. bytes: 0',
 'Duration: 5032 , Dest. byt

In [269]:
import time
start_time = time.time()

df[['protocol_type']].groupby(['protocol_type']).count().show()

print('Times to execute the request : %.3f s' % (time.time() - start_time))

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

Times to execute the request : 9.568 s


In [270]:
start_time = time.time()

df[(df['protocol_type'] == 'tcp') \
    & \
    (df['duration']>1000) \
    & \
    (df['dst_bytes'] == 0)].groupby(['protocol_type']).count().show()

print('Times to execute the request : %.3f s' % (time.time() - start_time))

+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+

Times to execute the request : 9.610 s


In [271]:
#Afficher les résultats


In [321]:
r = df[(df['protocol_type'] == 'tcp') \
    & \
    (df['duration']>1000) \
    & \
    (df['dst_bytes'] == 0)]
# r.show()
r.rdd.map(lambda r: 'Duration: {0} , Dest. bytes: {1}'.format(r['duration'], r['dst_bytes'])).take(4)

['Duration: 5057 , Dest. bytes: 0',
 'Duration: 5059 , Dest. bytes: 0',
 'Duration: 5051 , Dest. bytes: 0',
 'Duration: 5056 , Dest. bytes: 0']

In [338]:
from pyspark.sql.functions import udf

start_time = time.time()

def attack_or_normal_func(s):
    return "normal" if s == "normal." else "attack"

udf_attack_or_normal = udf(attack_or_normal_func, StringType())

df = df.withColumn('label',udf_attack_or_normal(df.interactions))
df.select('label').groupby('label').count().show()

print('Times to execute the request : %.3f s' % (time.time() - start_time))

+------+------+
| label| count|
+------+------+
|normal| 97278|
|attack|396743|
+------+------+

Times to execute the request : 9.527 s


In [334]:
df.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- interactions: string (nullable = true)
 |-- label: string (nullable = true)



In [339]:
start_time = time.time()

df.select(['label', 'protocol_type']).groupby(['label', 'protocol_type']).count().show()


print('Times to execute the request : %.3f s' % (time.time() - start_time))

+------+-------------+------+
| label|protocol_type| count|
+------+-------------+------+
|normal|          udp| 19177|
|normal|         icmp|  1288|
|normal|          tcp| 76813|
|attack|         icmp|282314|
|attack|          tcp|113252|
|attack|          udp|  1177|
+------+-------------+------+

Times to execute the request : 9.867 s
